# FlowMixer Implementation and Experiments for Times Series Forecasting - TENSORFLOW
## Supplementary Material

This notebook provides the complete implementation and experimental validation of the FlowMixer architecture presented in our paper. It contains the following key components:

### 1. Core Implementation
- FlowMixer neural architecture with time and feature mixing layers
- RevIN (Reversible Instance Normalization) implementation + TD-RevIn (Time Dependent)
- Custom loss functions and evaluation metrics
- Utility functions for data processing and model training

### 2. Experimental Validation
- Long-horizon time series forecasting experiments
- Benchmark comparisons across multiple datasets (ETT, Weather, Traffic, etc.)

- Visualization tools for model analysis and results presentation

### 3. Usage Instructions
To reproduce our experiments:
1. Ensure all required dependencies are installed (TensorFlow, NumPy, Pandas)
2. **Download the benchmark datasets to the `./data` directory**
3. Run the cells in sequence to train and evaluate the models
4. Use the visualization functions to analyze results

File names should end in csv: ETTh1.csv ETTm2.csv ..etc
Results are dislayed in tables at the end of training.

### Environment Requirements
- Python 3.7+
- TensorFlow 2.x
- NumPy
- Pandas
- Matplotlib
- scikit-learn

### References

This paper uses code from TSMixer paper (Chen et al. 2023), and its associated repository.

#### Please note This Notebook has results of the standard mixing.



# Main Code

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW, SGD
from sklearn.preprocessing import StandardScaler

# Reproduction
SEED = 888
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:
# @title DataLoader


"""Load raw data and generate time series dataset."""


#DATA_DIR = 'gs://time_series_datasets'
LOCAL_CACHE_DIR = './data/'


class DataLoader:
  """Generate data loader from raw data."""

  def __init__(
      self, data, batch_size, seq_len, pred_len, feature_type, target='OT'
  ):
    self.data = data
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.feature_type = feature_type
    self.target = target
    self.target_slice = slice(0, None)
    self.n_feature=1
    self._read_data()

  def _read_data(self):
    """Load raw data and split datasets."""

    # copy data from cloud storage if not exists
    if not os.path.isdir(LOCAL_CACHE_DIR):
      os.mkdir(LOCAL_CACHE_DIR)

    file_name = self.data + '.csv'
    cache_filepath = os.path.join(LOCAL_CACHE_DIR, file_name)
    #if not os.path.isfile(cache_filepath):
    #  tf.io.gfile.copy(
    #      os.path.join(DATA_DIR, file_name), cache_filepath, overwrite=True
    #  )

    df_raw = pd.read_csv(cache_filepath)

    # S: univariate-univariate, M: multivariate-multivariate, MS:
    # multivariate-univariate
    df = df_raw.set_index('date')
    if self.feature_type == 'S':
      df = df[[self.target]]
    elif self.feature_type == 'MS':
      target_idx = df.columns.get_loc(self.target)
      self.target_slice = slice(target_idx, target_idx + 1)

    # split train/valid/test
    n = len(df)
    if self.data.startswith('ETTm'):
      train_end = 12 * 30 * 24 * 4
      val_end = train_end + 4 * 30 * 24 * 4
      test_end = val_end + 4 * 30 * 24 * 4
    elif self.data.startswith('ETTh'):
      train_end = 12 * 30 * 24
      val_end = train_end + 4 * 30 * 24
      test_end = val_end + 4 * 30 * 24
    else:
      train_end = int(n * 0.7)
      val_end = n - int(n * 0.2)
      test_end = n
    train_df = df[:train_end]
    val_df = df[train_end - self.seq_len : val_end]
    test_df = df[val_end - self.seq_len : test_end]

    # standardize by training set
    self.scaler = StandardScaler()
    self.scaler.fit(train_df.values)

    def scale_df(df, scaler):
      data = scaler.transform(df.values)
      return pd.DataFrame(data, index=df.index, columns=df.columns)

    self.train_df = scale_df(train_df, self.scaler)
    self.val_df = scale_df(val_df, self.scaler)
    self.test_df = scale_df(test_df, self.scaler)
    self.n_feature = self.train_df.shape[-1]

  def _split_window(self, data):
    inputs = data[:, : self.seq_len, :]
    #labels = data[:, self.seq_len :, self.target_slice]
    labels = data[:, self.pred_len:self.seq_len+self.pred_len, self.target_slice]
    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.seq_len, None])
    #labels.set_shape([None, self.pred_len, None])
    labels.set_shape([None, self.seq_len, None])
    return inputs, labels

  def _make_dataset(self, data, shuffle=True):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=(self.seq_len + self.pred_len),
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=self.batch_size,
    )
    ds = ds.map(self._split_window)
    return ds

  def inverse_transform(self, data):
    return self.scaler.inverse_transform(data)

  def get_train(self, shuffle=True):
    return self._make_dataset(self.train_df, shuffle=shuffle)

  def get_val(self):
    return self._make_dataset(self.val_df, shuffle=False)

  def get_test(self):
    return self._make_dataset(self.test_df, shuffle=False)

In [ ]:
# @title MSEs

def mse_mod(y_true, y_pred, pred_len):
    """
    Custom MSE loss function that focuses on the last pred_len elements.

    Parameters:
    y_true (tf.Tensor): True values.
    y_pred (tf.Tensor): Predicted values.
    pred_len (int): Number of elements to focus on from the end.

    Returns:
    tf.Tensor: Computed MSE loss.
    """
    y_true_last = y_true[:, -pred_len , :]
    y_pred_last = y_pred[:, -pred_len , :]

    # Compute MSE on the last pred_len elements
    loss = tf.keras.losses.MeanSquaredError()(y_true_last, y_pred_last)


    return loss


In [ ]:
# @title FlowMixer

## REVIN / TD-REVIN
class RevNorm(tf.keras.layers.Layer):
    """Reversible Instance Normalization.
    This layer can normalize or denormalize data based on the mode specified.
    Attributes:
        axis (int): The axis over which to normalize the data.
        eps (float): A small constant added to variance to avoid division by zero.
        affine (bool): Whether to use affine transformation during normalization.
    """
    def __init__(self, axis=1, eps=1e-5, affine=True, revin=1):
        super().__init__()
        self.axis = axis
        self.eps = eps
        self.affine = affine
        self.affine_weight = None
        self.affine_bias = None
        self.mean = None
        self.stdev = None
        self.revin=revin

    def build(self, input_shape):
        if self.affine:
            if self.revin == 1:
              param_shape = (input_shape[-1],) # TYPE I Classic REVIN
            else:
              param_shape = (1,input_shape[-2],input_shape[-1]) # TYPE II TD-REVIN



            self.affine_weight = self.add_weight(
                name='affine_weight', shape=param_shape, initializer='ones', trainable=True
            )
            self.affine_bias = self.add_weight(
                name='affine_bias', shape=param_shape, initializer='zeros', trainable=False
            )


    def call(self, x, mode):

        if mode == 'norm':
            self.mean, self.stdev = self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else:
            raise NotImplementedError("Mode not supported, choose 'norm' or 'denorm'")
        return x

    def _get_statistics(self, x):
        mean = tf.stop_gradient(
            tf.reduce_mean(x, axis=self.axis, keepdims=True)
        )
        stdev = tf.stop_gradient(
            tf.sqrt(tf.math.reduce_variance(x, axis=self.axis, keepdims=True) + self.eps)
        )
        return mean, stdev

    def _normalize(self, x):
        x = (x - self.mean) / self.stdev
        if self.affine:
            x = x * self.affine_weight + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = (x - self.affine_bias) / self.affine_weight
        x = x * self.stdev + self.mean
        return x



# OPTIONAL
class NoiseInjectionLayer(tf.keras.layers.Layer):
    def __init__(self, noise_factor=0.01, axis=1, noise_type='gaussian',
                 trainable=False, apply_during_inference=False):
        """
        Initializes the enhanced noise injection layer.
        :param noise_factor: The factor of the signal energy by which the noise is scaled.
        :param axis: The axis along which to calculate the signal energy and inject noise.
        :param noise_type: The type of noise to inject ('gaussian' or 'uniform').
        :param trainable: Whether the noise_factor should be trainable.
        :param apply_during_inference: Whether to apply noise during inference.
        """
        super(NoiseInjectionLayer, self).__init__()
        self.axis = axis
        self.noise_type = noise_type
        self.apply_during_inference = apply_during_inference

        if trainable:
            self.noise_factor = tf.Variable(noise_factor,
                                            constraint=lambda x: tf.clip_by_value(x, 0, 1),
                                            trainable=True)
        else:
            self.noise_factor = noise_factor

    def call(self, inputs, training=None):
        if training is None:
            training = tf.keras.backend.learning_phase()

        if training or self.apply_during_inference:
            # Calculate the signal energy along the specified axis
            signal_energy = tf.reduce_mean(tf.square(inputs), axis=self.axis, keepdims=True)

            # Calculate the standard deviation of the noise from signal energy
            noise_std = tf.sqrt(signal_energy) * self.noise_factor

            # Generate noise
            if self.noise_type == 'gaussian':
                noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=1.0)
            elif self.noise_type == 'uniform':
                noise = tf.random.uniform(shape=tf.shape(inputs), minval=-1.0, maxval=1.0)
            else:
                raise ValueError(f"Unsupported noise type: {self.noise_type}")

            # Scale and add noise to the input
            return inputs + noise * noise_std
        else:
            return inputs

    def get_config(self):
        config = super(NoiseInjectionLayer, self).get_config()
        config.update({
            'noise_factor': self.noise_factor.numpy() if isinstance(self.noise_factor, tf.Variable) else self.noise_factor,
            'axis': self.axis,
            'noise_type': self.noise_type,
            'trainable': isinstance(self.noise_factor, tf.Variable),
            'apply_during_inference': self.apply_during_inference
        })
        return config


## MIXER
class MixingModule(tf.keras.layers.Layer):
    def __init__(self, pred_len, **kwargs):
        super(MixingModule, self).__init__(**kwargs)
        self.pred_len = pred_len
        self.wt = None

    def wt_initializer(self, shape, dtype=tf.float32):
        seq_len = shape[0]
        initial_wt = tf.random.uniform(shape, minval=-0.1, maxval=0.1, dtype=dtype)

        # Create the desired structure
        for i in range(seq_len - self.pred_len):
            initial_wt = tf.tensor_scatter_nd_update(
                initial_wt,
                [[i , i+self.pred_len]],
                [1.0]
            )

        return initial_wt


    def build(self, input_shape):
        # Initialize weights for feature transformation
        self.W = self.add_weight(name='W',
                                  shape=(input_shape[-2], input_shape[-2]),
                                  #initializer=self.wt_initializer,
                                  initializer='glorot_normal',
                                  trainable=True)


        self.scale_t = self.add_weight(name='scale_t', shape=(), initializer='zeros', trainable=True)
        self.scale_f = self.add_weight(name='scale_f', shape=(), initializer='ones', trainable=True)

        self.K = self.add_weight(name='K',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.Q = self.add_weight(name='Q',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)

        super().build(input_shape)

    def call(self, inputs):
        x = inputs

        Wt = self.scale_t * tf.eye(inputs.shape[-2]) + self.W * self.W



        Wf_T = tf.eye(inputs.shape[-1])+self.scale_f*self.scale_f*tf.nn.softmax(self.Q@tf.transpose(self.K)/tf.sqrt(1e-16+inputs.shape[-1]),axis=0)
        Wf_T = Wf_T/(1.0+self.scale_f*self.scale_f)


        x= x + Wt @ x @ Wf_T

        return x

    def defWt(self):
        Wt = self.scale_t * tf.eye(self.wt.shape[-1]) + self.wt * self.wt
        return Wt

    def defWf(self):
        Wf = tf.eye(self.K.shape[-1])+self.scale_f*self.scale_f*tf.nn.softmax(self.Q@tf.transpose(self.K)/tf.sqrt(1e-16+self.K.shape[-1]),axis=0)
        Wf = Wf/(1.0+self.scale_f*self.scale_f)
        return Wf


## FlowMixer
class FlowMixer(tf.keras.Model):
    def __init__(self, seq_len, nb_features, pred_len, dropout_rate, noise_factor, revin=1):
        super(FlowMixer, self).__init__()
        self.seq_len = seq_len
        self.nb_features = nb_features
        self.dropout_rate=dropout_rate
        self.noise_factor = noise_factor
        self.revin=revin
        self.pred_len = pred_len




        self.noise_layer = NoiseInjectionLayer(noise_factor=self.noise_factor, axis=1, noise_type='uniform',
                                            trainable=False, apply_during_inference=False)# Optional


        self.mixer_layer = MixingModule(seq_len)

        self.drop_layer=tf.keras.layers.Dropout(self.dropout_rate)

        self.revin_layer = RevNorm(axis=-2, revin=self.revin)





    def call(self, inputs, training=True):

        x = inputs
        x = self.revin_layer(x, mode='norm')

        x = self.drop_layer(x)
        if training:
            x = self.noise_layer(x)

        x = self.mixer_layer(x)
        x = self.revin_layer(x, mode='denorm')

        return x


In [ ]:
# @title Train and Evaluate


def custom_evaluation(model, data, pred_len):
    mse = tf.keras.metrics.MeanSquaredError()
    mae = tf.keras.metrics.MeanAbsoluteError()

    for inputs, labels in data:
        forecasts = model(inputs, training=False)
        labels_last = labels[:, -pred_len:, :]
        forecasts_last = forecasts[:, -pred_len:, :]
        mse.update_state(labels_last, forecasts_last)
        mae.update_state(labels_last, forecasts_last)

    return mse.result().numpy(), mae.result().numpy()

class CustomEvaluation(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, pred_len):
        super(CustomEvaluation, self).__init__()
        self.validation_data = validation_data
        self.pred_len = pred_len
        self.best_mse = float('inf')
        self.best_mae = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        mse, mae = custom_evaluation(self.model, self.validation_data, self.pred_len)# self.validation
        logs['val_custom_mse'] = mse
        logs['val_custom_mae'] = mae
        if mse < self.best_mse:
            self.best_mse = mse
            self.best_mae = mae

def train_and_evaluate(data_loader, seq_len, pred_len, dropout_rate, noise_factor=0.00, revin=2, epochs=100, learning_rate=1e-4, mopt='sgd'):
    train_data = data_loader.get_train()
    val_data = data_loader.get_val()
    test_data = data_loader.get_test()
    nb_features = data_loader.n_feature

    model = FlowMixer(seq_len, nb_features, pred_len, dropout_rate, noise_factor, revin=revin)
    if mopt=='sgd':
      optimizer = SGD(learning_rate=learning_rate, weight_decay=1e-6, momentum=0.9)
    elif mopt=='adamw':
      optimizer = AdamW(learning_rate=learning_rate, weight_decay=1e-6)
    else:
      optimizer = Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae'])

    # Create checkpoint directory
    if not os.path.exists('./model_checkpoints'):
        os.makedirs('./model_checkpoints')

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_custom_mse',
        patience=24,#100,#24
        mode='min',
        restore_best_weights=True  # This will restore best weights automatically
    )
    reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_mse',
        factor=0.2,
        patience=8,#100,#8
        verbose=1,
        mode='min'
    )
    #custom_eval_callback = CustomEvaluation(test_data, pred_len)# cgange it back
    custom_eval_callback = CustomEvaluation(val_data, pred_len)# change it back
    # Add checkpoint callback
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'./model_checkpoints/model_{seq_len}_{pred_len}_{dropout_rate}.weights.h5',
        monitor='val_mse',
        save_best_only=True,
        mode='min',
        save_weights_only=True,
        verbose=0
    )

    model.fit(
        train_data,
        epochs=epochs,
        validation_data=val_data,
        callbacks=[early_stop_callback, reduce_lr_callback, custom_eval_callback, checkpoint_callback],
        verbose=2,
        shuffle=True
    )

    test_mse, test_mae = custom_evaluation(model, test_data, pred_len)
    return custom_eval_callback.best_mse, custom_eval_callback.best_mae, test_mse, test_mae, model

def run_experiments(data_name='ETTh1', seq_len_=1024, horizons=[96,192,336,720], dropout_rates=[0.1, 0.2, 0.3], batch_size=32, revin=1, epochs=100, learning_rate=1e-4, mopt='sgd'):
    results = []
    models = {}

    for pred_len in horizons:
        if pred_len > seq_len_:
            seq_len = pred_len # PADDING
        else:
            seq_len = seq_len_

        data_loader = DataLoader(data=data_name, feature_type='M', batch_size=batch_size,
                                 seq_len=seq_len, pred_len=pred_len) # M for Multivariate

        for dropout_rate in dropout_rates:
            print(f"Running experiment: horizon={pred_len}, dropout_rate={dropout_rate}")
            val_mse, val_mae, test_mse, test_mae, model = train_and_evaluate(data_loader, seq_len, pred_len, dropout_rate, revin=revin, epochs=epochs, learning_rate=learning_rate, mopt=mopt)

            results.append({
                'horizon': pred_len,
                'dropout_rate': dropout_rate,
                'val_mse': val_mse,
                'val_mae': val_mae,
                'test_mse': test_mse,
                'test_mae': test_mae
            })

            # Store the model
            models[(pred_len, dropout_rate)] = model

            results_dir = './flowmixer_results'
            if not os.path.exists(results_dir):
                os.makedirs(results_dir)
            results_file = os.path.join(results_dir, f'{data_name}_experiment_results.csv')
            #results_gdir = '/content/drive/MyDrive/FlowMixer/flowmixer_results_nmi'
            #results_gfile = os.path.join(results_gdir, f'{data_name}_experiment_results.csv')
            # Save results after each experiment
            df = pd.DataFrame(results)
            df.to_csv(results_file, index=False)
            #df.to_csv(results_gfile, index=False)


    return results, models


def display_results_tables(results):
   """
   Display MSE and MAE results in formatted tables.

   Args:
       results: List of dictionaries containing results with keys:
               horizon, dropout_rate, val_mse, val_mae, test_mse, test_mae
   """
   # Create DataFrame from results
   df = pd.DataFrame(results)

   # Create pivot tables
   mse_table = pd.pivot_table(df, values='test_mse', index='horizon', columns='dropout_rate')
   mae_table = pd.pivot_table(df, values='test_mae', index='horizon', columns='dropout_rate')

   # Format column headers to show dropout rates as percentages
   mse_table.columns = [f'DR={x:.0%}' for x in mse_table.columns]
   mae_table.columns = [f'DR={x:.0%}' for x in mae_table.columns]

   # Rename index
   mse_table.index.name = 'Horizon'
   mae_table.index.name = 'Horizon'

   # Display tables with nice formatting
   print("\nMSE Results:")
   print("=" * 50)
   print(mse_table.round(4))

   print("\nMAE Results:")
   print("=" * 50)
   print(mae_table.round(4))

# Usage:
#display_results_tables(results[0])  #

# Example usage:
# results, models = run_experiments()
#
# # To access a specific model:
# model = models[(96, 0.1)]  # Gets the model for horizon=96 and dropout_rate=0.1
#
# # You can now use this model for further predictions or analysis

# Results

## ETTh1

In [ ]:
# Run the experiments
data_name='ETTh1'
results = run_experiments(data_name,horizons=[96,192,336,720], dropout_rates=[0.0, 0.1, 0.2, 0.3], revin=1, seq_len_=1024, learning_rate=1e-1, mopt='sgd')

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")



Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


236/236 - 5s - 22ms/step - loss: 0.4244 - mae: 0.4586 - mse: 0.4244 - val_loss: 0.7782 - val_mae: 0.6197 - val_mse: 0.7782 - learning_rate: 0.1000 - val_custom_mse: 0.9545 - val_custom_mae: 0.6896
Epoch 2/100
236/236 - 2s - 9ms/step - loss: 0.4035 - mae: 0.4482 - mse: 0.4035 - val_loss: 0.7290 - val_mae: 0.5983 - val_mse: 0.7290 - learning_rate: 0.1000 - val_custom_mse: 0.9243 - val_custom_mae: 0.6760
Epoch 3/100
236/236 - 2s - 9ms/step - loss: 0.3830 - mae: 0.4364 - mse: 0.3830 - val_loss: 0.6764 - val_mae: 0.5751 - val_mse: 0.6764 - learning_rate: 0.1000 - val_custom_mse: 0.9031 - val_custom_mae: 0.6662
Epoch 4/100
236/236 - 2s - 8ms/step - loss: 0.3580 - mae: 0.4216 - mse: 0.3580 - val_loss: 0.6162 - val_mae: 0.5483 - val_mse: 0.6162 - learning_rate: 0.1000 - val_custom_mse: 0.8869 - val_custom_mae: 0.6590
Epoch 5/100
236/236 - 2s - 8ms/step - loss: 0.3271 - mae: 0.4024 - mse: 0.3271 - val_loss: 0.5442 - val_mae: 0.5141 - val_mse: 0.5442 - learning_rate: 0.1000 - val_custom_mse: 0.8

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_18', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


236/236 - 6s - 27ms/step - loss: 0.4381 - mae: 0.4675 - mse: 0.4381 - val_loss: 0.7752 - val_mae: 0.6214 - val_mse: 0.7752 - learning_rate: 0.1000 - val_custom_mse: 0.9619 - val_custom_mae: 0.6949
Epoch 2/100
236/236 - 2s - 9ms/step - loss: 0.4117 - mae: 0.4540 - mse: 0.4117 - val_loss: 0.7301 - val_mae: 0.6031 - val_mse: 0.7301 - learning_rate: 0.1000 - val_custom_mse: 0.9471 - val_custom_mae: 0.6885
Epoch 3/100
236/236 - 2s - 9ms/step - loss: 0.3879 - mae: 0.4404 - mse: 0.3879 - val_loss: 0.6734 - val_mae: 0.5781 - val_mse: 0.6734 - learning_rate: 0.1000 - val_custom_mse: 0.9249 - val_custom_mae: 0.6782
Epoch 4/100
236/236 - 2s - 8ms/step - loss: 0.3594 - mae: 0.4234 - mse: 0.3594 - val_loss: 0.6103 - val_mae: 0.5503 - val_mse: 0.6103 - learning_rate: 0.1000 - val_custom_mse: 0.9128 - val_custom_mae: 0.6731
Epoch 5/100
236/236 - 2s - 8ms/step - loss: 0.3249 - mae: 0.4019 - mse: 0.3249 - val_loss: 0.5283 - val_mae: 0.5089 - val_mse: 0.5283 - learning_rate: 0.1000 - val_custom_mse: 0.8

...

216/216 - 6s - 28ms/step - loss: 0.7128 - mae: 0.6186 - mse: 0.7128 - val_loss: 1.5169 - val_mae: 0.9099 - val_mse: 1.5169 - learning_rate: 0.1000 - val_custom_mse: 1.6649 - val_custom_mae: 0.9558
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.6576 - mae: 0.5956 - mse: 0.6576 - val_loss: 1.4321 - val_mae: 0.8788 - val_mse: 1.4321 - learning_rate: 0.1000 - val_custom_mse: 1.6077 - val_custom_mae: 0.9346
Epoch 3/100
216/216 - 2s - 8ms/step - loss: 0.6221 - mae: 0.5771 - mse: 0.6221 - val_loss: 1.3526 - val_mae: 0.8481 - val_mse: 1.3526 - learning_rate: 0.1000 - val_custom_mse: 1.5564 - val_custom_mae: 0.9150
Epoch 4/100
216/216 - 2s - 8ms/step - loss: 0.5865 - mae: 0.5575 - mse: 0.5865 - val_loss: 1.2621 - val_mae: 0.8116 - val_mse: 1.2621 - learning_rate: 0.1000 - val_custom_mse: 1.4906 - val_custom_mae: 0.8894
Epoch 5/100
216/216 - 2s - 7ms/step - loss: 0.5513 - mae: 0.5366 - mse: 0.5513 - val_loss: 1.1845 - val_mae: 0.7758 - val_mse: 1.1845 - learning_rate: 0.1000 - val_custom_mse: 1.4

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_31', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


216/216 - 6s - 28ms/step - loss: 0.7204 - mae: 0.6213 - mse: 0.7204 - val_loss: 1.5267 - val_mae: 0.9129 - val_mse: 1.5267 - learning_rate: 0.1000 - val_custom_mse: 1.6848 - val_custom_mae: 0.9624
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.6606 - mae: 0.5968 - mse: 0.6606 - val_loss: 1.4285 - val_mae: 0.8777 - val_mse: 1.4285 - learning_rate: 0.1000 - val_custom_mse: 1.6051 - val_custom_mae: 0.9339
Epoch 3/100
216/216 - 2s - 8ms/step - loss: 0.6230 - mae: 0.5775 - mse: 0.6230 - val_loss: 1.3524 - val_mae: 0.8477 - val_mse: 1.3524 - learning_rate: 0.1000 - val_custom_mse: 1.5583 - val_custom_mae: 0.9155
Epoch 4/100
216/216 - 2s - 8ms/step - loss: 0.5863 - mae: 0.5573 - mse: 0.5863 - val_loss: 1.2568 - val_mae: 0.8094 - val_mse: 1.2568 - learning_rate: 0.1000 - val_custom_mse: 1.4864 - val_custom_mae: 0.8877
Epoch 5/100
216/216 - 2s - 8ms/step - loss: 0.5504 - mae: 0.5359 - mse: 0.5504 - val_loss: 1.1785 - val_mae: 0.7734 - val_mse: 1.1785 - learning_rate: 0.1000 - val_custom_mse: 1.4

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_32', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


216/216 - 6s - 28ms/step - loss: 0.7273 - mae: 0.6241 - mse: 0.7273 - val_loss: 1.5220 - val_mae: 0.9120 - val_mse: 1.5220 - learning_rate: 0.1000 - val_custom_mse: 1.6767 - val_custom_mae: 0.9607
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.6637 - mae: 0.5980 - mse: 0.6637 - val_loss: 1.4291 - val_mae: 0.8780 - val_mse: 1.4291 - learning_rate: 0.1000 - val_custom_mse: 1.6053 - val_custom_mae: 0.9341
Epoch 3/100
216/216 - 2s - 7ms/step - loss: 0.6241 - mae: 0.5781 - mse: 0.6241 - val_loss: 1.3508 - val_mae: 0.8476 - val_mse: 1.3508 - learning_rate: 0.1000 - val_custom_mse: 1.5557 - val_custom_mae: 0.9151
Epoch 4/100
216/216 - 2s - 7ms/step - loss: 0.5864 - mae: 0.5574 - mse: 0.5864 - val_loss: 1.2565 - val_mae: 0.8089 - val_mse: 1.2565 - learning_rate: 0.1000 - val_custom_mse: 1.4856 - val_custom_mae: 0.8871
Epoch 5/100
216/216 - 2s - 7ms/step - loss: 0.5503 - mae: 0.5358 - mse: 0.5503 - val_loss: 1.1874 - val_mae: 0.7759 - val_mse: 1.1874 - learning_rate: 0.1000 - val_custom_mse: 1.4

## ETTh2

In [ ]:
# Run the experiments
data_name='ETTh2'
results = run_experiments(data_name,horizons=[96,192,336,720], dropout_rates=[0.0, 0.1, 0.2, 0.3], revin=1, seq_len_=1024, learning_rate=1e-1, mopt='sgd')

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_33', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


236/236 - 5s - 22ms/step - loss: 0.4728 - mae: 0.4123 - mse: 0.4728 - val_loss: 0.2741 - val_mae: 0.3782 - val_mse: 0.2741 - learning_rate: 0.1000 - val_custom_mse: 0.3736 - val_custom_mae: 0.4494
Epoch 2/100
236/236 - 2s - 9ms/step - loss: 0.4247 - mae: 0.3925 - mse: 0.4247 - val_loss: 0.2386 - val_mae: 0.3545 - val_mse: 0.2386 - learning_rate: 0.1000 - val_custom_mse: 0.3510 - val_custom_mae: 0.4352
Epoch 3/100
236/236 - 2s - 8ms/step - loss: 0.3723 - mae: 0.3689 - mse: 0.3723 - val_loss: 0.2079 - val_mae: 0.3292 - val_mse: 0.2079 - learning_rate: 0.1000 - val_custom_mse: 0.3381 - val_custom_mae: 0.4228
Epoch 4/100
236/236 - 2s - 8ms/step - loss: 0.3179 - mae: 0.3402 - mse: 0.3179 - val_loss: 0.1724 - val_mae: 0.2990 - val_mse: 0.1724 - learning_rate: 0.1000 - val_custom_mse: 0.3141 - val_custom_mae: 0.4042
Epoch 5/100
236/236 - 2s - 8ms/step - loss: 0.2600 - mae: 0.3092 - mse: 0.2600 - val_loss: 0.1399 - val_mae: 0.2679 - val_mse: 0.1399 - learning_rate: 0.1000 - val_custom_mse: 0.2

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_34', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


236/236 - 6s - 25ms/step - loss: 0.4793 - mae: 0.4169 - mse: 0.4793 - val_loss: 0.2760 - val_mae: 0.3800 - val_mse: 0.2760 - learning_rate: 0.1000 - val_custom_mse: 0.3903 - val_custom_mae: 0.4583
Epoch 2/100
236/236 - 2s - 8ms/step - loss: 0.4266 - mae: 0.3946 - mse: 0.4266 - val_loss: 0.2369 - val_mae: 0.3522 - val_mse: 0.2369 - learning_rate: 0.1000 - val_custom_mse: 0.3495 - val_custom_mae: 0.4321
Epoch 3/100
236/236 - 2s - 8ms/step - loss: 0.3735 - mae: 0.3698 - mse: 0.3735 - val_loss: 0.2012 - val_mae: 0.3229 - val_mse: 0.2012 - learning_rate: 0.1000 - val_custom_mse: 0.3181 - val_custom_mae: 0.4079
Epoch 4/100
236/236 - 2s - 8ms/step - loss: 0.3162 - mae: 0.3418 - mse: 0.3162 - val_loss: 0.1686 - val_mae: 0.2953 - val_mse: 0.1686 - learning_rate: 0.1000 - val_custom_mse: 0.3036 - val_custom_mae: 0.3965
Epoch 5/100
236/236 - 2s - 8ms/step - loss: 0.2600 - mae: 0.3105 - mse: 0.2600 - val_loss: 0.1391 - val_mae: 0.2673 - val_mse: 0.1391 - learning_rate: 0.1000 - val_custom_mse: 0.2

...

216/216 - 6s - 28ms/step - loss: 0.8575 - mae: 0.5972 - mse: 0.8575 - val_loss: 0.6813 - val_mae: 0.5816 - val_mse: 0.6813 - learning_rate: 0.1000 - val_custom_mse: 0.8454 - val_custom_mae: 0.6576
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.7746 - mae: 0.5692 - mse: 0.7746 - val_loss: 0.6530 - val_mae: 0.5668 - val_mse: 0.6530 - learning_rate: 0.1000 - val_custom_mse: 0.8217 - val_custom_mae: 0.6483
Epoch 3/100
216/216 - 2s - 8ms/step - loss: 0.7376 - mae: 0.5525 - mse: 0.7376 - val_loss: 0.6244 - val_mae: 0.5474 - val_mse: 0.6244 - learning_rate: 0.1000 - val_custom_mse: 0.7999 - val_custom_mae: 0.6346
Epoch 4/100
216/216 - 2s - 7ms/step - loss: 0.7022 - mae: 0.5350 - mse: 0.7022 - val_loss: 0.6441 - val_mae: 0.5525 - val_mse: 0.6441 - learning_rate: 0.1000 - val_custom_mse: 0.8427 - val_custom_mae: 0.6544
Epoch 5/100
216/216 - 2s - 8ms/step - loss: 0.6785 - mae: 0.5221 - mse: 0.6785 - val_loss: 0.6198 - val_mae: 0.5363 - val_mse: 0.6198 - learning_rate: 0.1000 - val_custom_mse: 0.8

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_47', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


216/216 - 6s - 28ms/step - loss: 0.8590 - mae: 0.5977 - mse: 0.8590 - val_loss: 0.6819 - val_mae: 0.5811 - val_mse: 0.6819 - learning_rate: 0.1000 - val_custom_mse: 0.8483 - val_custom_mae: 0.6584
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.7793 - mae: 0.5696 - mse: 0.7793 - val_loss: 0.7215 - val_mae: 0.6039 - val_mse: 0.7215 - learning_rate: 0.1000 - val_custom_mse: 0.9040 - val_custom_mae: 0.6903
Epoch 3/100
216/216 - 2s - 7ms/step - loss: 0.7374 - mae: 0.5521 - mse: 0.7374 - val_loss: 0.6613 - val_mae: 0.5646 - val_mse: 0.6613 - learning_rate: 0.1000 - val_custom_mse: 0.8505 - val_custom_mae: 0.6581
Epoch 4/100
216/216 - 2s - 7ms/step - loss: 0.7038 - mae: 0.5357 - mse: 0.7038 - val_loss: 0.6250 - val_mae: 0.5429 - val_mse: 0.6250 - learning_rate: 0.1000 - val_custom_mse: 0.8163 - val_custom_mae: 0.6414
Epoch 5/100
216/216 - 2s - 7ms/step - loss: 0.6796 - mae: 0.5224 - mse: 0.6796 - val_loss: 0.6343 - val_mae: 0.5439 - val_mse: 0.6343 - learning_rate: 0.1000 - val_custom_mse: 0.8

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_48', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


216/216 - 6s - 27ms/step - loss: 0.8633 - mae: 0.5993 - mse: 0.8633 - val_loss: 0.7172 - val_mae: 0.5975 - val_mse: 0.7172 - learning_rate: 0.1000 - val_custom_mse: 0.8863 - val_custom_mae: 0.6738
Epoch 2/100
216/216 - 2s - 8ms/step - loss: 0.7684 - mae: 0.5666 - mse: 0.7684 - val_loss: 0.6442 - val_mae: 0.5629 - val_mse: 0.6442 - learning_rate: 0.1000 - val_custom_mse: 0.8090 - val_custom_mae: 0.6426
Epoch 3/100
216/216 - 2s - 7ms/step - loss: 0.7407 - mae: 0.5536 - mse: 0.7407 - val_loss: 0.6472 - val_mae: 0.5584 - val_mse: 0.6472 - learning_rate: 0.1000 - val_custom_mse: 0.8307 - val_custom_mae: 0.6493
Epoch 4/100
216/216 - 2s - 7ms/step - loss: 0.7061 - mae: 0.5370 - mse: 0.7061 - val_loss: 0.6345 - val_mae: 0.5477 - val_mse: 0.6345 - learning_rate: 0.1000 - val_custom_mse: 0.8290 - val_custom_mae: 0.6475
Epoch 5/100
216/216 - 2s - 7ms/step - loss: 0.6813 - mae: 0.5235 - mse: 0.6813 - val_loss: 0.6397 - val_mae: 0.5469 - val_mse: 0.6397 - learning_rate: 0.1000 - val_custom_mse: 0.8

## ETTm1

In [ ]:
# Run the experiments
data_name='ETTm1'
results = run_experiments(data_name,horizons=[96,192,336,720], dropout_rates=[0.0, 0.1, 0.2, 0.3], seq_len_=1024, revin=2,learning_rate=1e-3, mopt='adamw')

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


1046/1046 - 14s - 13ms/step - loss: 0.1296 - mae: 0.2149 - mse: 0.1296 - val_loss: 0.0547 - val_mae: 0.1264 - val_mse: 0.0547 - learning_rate: 0.0010 - val_custom_mse: 0.3998 - val_custom_mae: 0.4218
Epoch 2/100
1046/1046 - 8s - 8ms/step - loss: 0.0292 - mae: 0.0769 - mse: 0.0292 - val_loss: 0.0398 - val_mae: 0.0721 - val_mse: 0.0398 - learning_rate: 0.0010 - val_custom_mse: 0.3998 - val_custom_mae: 0.4222
Epoch 3/100
1046/1046 - 8s - 8ms/step - loss: 0.0249 - mae: 0.0525 - mse: 0.0249 - val_loss: 0.0381 - val_mae: 0.0583 - val_mse: 0.0381 - learning_rate: 0.0010 - val_custom_mse: 0.3980 - val_custom_mae: 0.4204
Epoch 4/100
1046/1046 - 8s - 8ms/step - loss: 0.0242 - mae: 0.0447 - mse: 0.0242 - val_loss: 0.0375 - val_mae: 0.0529 - val_mse: 0.0375 - learning_rate: 0.0010 - val_custom_mse: 0.3952 - val_custom_mae: 0.4179
Epoch 5/100
1046/1046 - 8s - 8ms/step - loss: 0.0239 - mae: 0.0414 - mse: 0.0239 - val_loss: 0.0371 - val_mae: 0.0505 - val_mse: 0.0371 - learning_rate: 0.0010 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1046/1046 - 13s - 13ms/step - loss: 0.1590 - mae: 0.2489 - mse: 0.1590 - val_loss: 0.0600 - val_mae: 0.1361 - val_mse: 0.0600 - learning_rate: 0.0010 - val_custom_mse: 0.3998 - val_custom_mae: 0.4209
Epoch 2/100
1046/1046 - 8s - 8ms/step - loss: 0.0490 - mae: 0.1280 - mse: 0.0490 - val_loss: 0.0502 - val_mae: 0.1104 - val_mse: 0.0502 - learning_rate: 0.0010 - val_custom_mse: 0.3946 - val_custom_mae: 0.4170
Epoch 3/100
1046/1046 - 8s - 8ms/step - loss: 0.0442 - mae: 0.1144 - mse: 0.0442 - val_loss: 0.0491 - val_mae: 0.1072 - val_mse: 0.0491 - learning_rate: 0.0010 - val_custom_mse: 0.3919 - val_custom_mae: 0.4165
Epoch 4/100
1046/1046 - 8s - 8ms/step - loss: 0.0433 - mae: 0.1118 - mse: 0.0433 - val_loss: 0.0483 - val_mae: 0.1058 - val_mse: 0.0483 - learning_rate: 0.0010 - val_custom_mse: 0.3866 - val_custom_mae: 0.4113
Epoch 5/100
1046/1046 - 8s - 7ms/step - loss: 0.0429 - mae: 0.1109 - mse: 0.0429 - val_loss: 0.0482 - val_mae: 0.1052 - val_mse: 0.0482 - learning_rate: 0.0010 - val_cust

...

1026/1026 - 12s - 12ms/step - loss: 0.4461 - mae: 0.4622 - mse: 0.4461 - val_loss: 0.6722 - val_mae: 0.4997 - val_mse: 0.6722 - learning_rate: 0.0010 - val_custom_mse: 0.9367 - val_custom_mae: 0.6461
Epoch 2/100
1026/1026 - 7s - 7ms/step - loss: 0.2891 - mae: 0.3457 - mse: 0.2891 - val_loss: 0.6512 - val_mae: 0.4758 - val_mse: 0.6512 - learning_rate: 0.0010 - val_custom_mse: 0.9171 - val_custom_mae: 0.6341
Epoch 3/100
1026/1026 - 7s - 7ms/step - loss: 0.2800 - mae: 0.3329 - mse: 0.2800 - val_loss: 0.6450 - val_mae: 0.4698 - val_mse: 0.6450 - learning_rate: 0.0010 - val_custom_mse: 0.9099 - val_custom_mae: 0.6308
Epoch 4/100
1026/1026 - 7s - 7ms/step - loss: 0.2783 - mae: 0.3302 - mse: 0.2783 - val_loss: 0.6455 - val_mae: 0.4686 - val_mse: 0.6455 - learning_rate: 0.0010 - val_custom_mse: 0.9113 - val_custom_mae: 0.6308
Epoch 5/100
1026/1026 - 7s - 7ms/step - loss: 0.2776 - mae: 0.3293 - mse: 0.2776 - val_loss: 0.6465 - val_mae: 0.4679 - val_mse: 0.6465 - learning_rate: 0.0010 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1026/1026 - 12s - 12ms/step - loss: 0.4786 - mae: 0.4813 - mse: 0.4786 - val_loss: 0.6594 - val_mae: 0.4963 - val_mse: 0.6594 - learning_rate: 0.0010 - val_custom_mse: 0.9188 - val_custom_mae: 0.6416
Epoch 2/100
1026/1026 - 8s - 7ms/step - loss: 0.2940 - mae: 0.3528 - mse: 0.2940 - val_loss: 0.6455 - val_mae: 0.4783 - val_mse: 0.6455 - learning_rate: 0.0010 - val_custom_mse: 0.9066 - val_custom_mae: 0.6332
Epoch 3/100
1026/1026 - 7s - 7ms/step - loss: 0.2842 - mae: 0.3403 - mse: 0.2842 - val_loss: 0.6432 - val_mae: 0.4755 - val_mse: 0.6432 - learning_rate: 0.0010 - val_custom_mse: 0.9044 - val_custom_mae: 0.6320
Epoch 4/100
1026/1026 - 8s - 7ms/step - loss: 0.2830 - mae: 0.3386 - mse: 0.2830 - val_loss: 0.6421 - val_mae: 0.4739 - val_mse: 0.6421 - learning_rate: 0.0010 - val_custom_mse: 0.9030 - val_custom_mae: 0.6306
Epoch 5/100
1026/1026 - 7s - 7ms/step - loss: 0.2825 - mae: 0.3382 - mse: 0.2825 - val_loss: 0.6435 - val_mae: 0.4739 - val_mse: 0.6435 - learning_rate: 0.0010 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_15', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1026/1026 - 13s - 13ms/step - loss: 0.5193 - mae: 0.5017 - mse: 0.5193 - val_loss: 0.6596 - val_mae: 0.4968 - val_mse: 0.6596 - learning_rate: 0.0010 - val_custom_mse: 0.9179 - val_custom_mae: 0.6409
Epoch 2/100
1026/1026 - 8s - 7ms/step - loss: 0.2983 - mae: 0.3578 - mse: 0.2983 - val_loss: 0.6461 - val_mae: 0.4816 - val_mse: 0.6461 - learning_rate: 0.0010 - val_custom_mse: 0.9052 - val_custom_mae: 0.6341
Epoch 3/100
1026/1026 - 8s - 7ms/step - loss: 0.2882 - mae: 0.3457 - mse: 0.2882 - val_loss: 0.6422 - val_mae: 0.4787 - val_mse: 0.6422 - learning_rate: 0.0010 - val_custom_mse: 0.9003 - val_custom_mae: 0.6315
Epoch 4/100
1026/1026 - 7s - 7ms/step - loss: 0.2872 - mae: 0.3445 - mse: 0.2872 - val_loss: 0.6432 - val_mae: 0.4793 - val_mse: 0.6432 - learning_rate: 0.0010 - val_custom_mse: 0.9019 - val_custom_mae: 0.6327
Epoch 5/100
1026/1026 - 8s - 7ms/step - loss: 0.2867 - mae: 0.3442 - mse: 0.2867 - val_loss: 0.6432 - val_mae: 0.4786 - val_mse: 0.6432 - learning_rate: 0.0010 - val_cust

## ETTm2

In [ ]:
# Run the experiments
data_name='ETTm2'
results = run_experiments(data_name,horizons=[96,192,336,720], dropout_rates=[0.0, 0.1, 0.2, 0.3], revin=1, seq_len_=1204,  learning_rate=1e-1, mopt='sgd')

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_16', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


1040/1040 - 13s - 13ms/step - loss: 0.2435 - mae: 0.2858 - mse: 0.2435 - val_loss: 0.1077 - val_mae: 0.2348 - val_mse: 0.1077 - learning_rate: 0.1000 - val_custom_mse: 0.1599 - val_custom_mae: 0.2840
Epoch 2/100
1040/1040 - 8s - 8ms/step - loss: 0.1691 - mae: 0.2459 - mse: 0.1691 - val_loss: 0.0774 - val_mae: 0.1971 - val_mse: 0.0774 - learning_rate: 0.1000 - val_custom_mse: 0.1484 - val_custom_mae: 0.2739
Epoch 3/100
1040/1040 - 8s - 8ms/step - loss: 0.1070 - mae: 0.2015 - mse: 0.1070 - val_loss: 0.0582 - val_mae: 0.1675 - val_mse: 0.0582 - learning_rate: 0.1000 - val_custom_mse: 0.1394 - val_custom_mae: 0.2652
Epoch 4/100
1040/1040 - 8s - 8ms/step - loss: 0.0753 - mae: 0.1699 - mse: 0.0753 - val_loss: 0.0489 - val_mae: 0.1502 - val_mse: 0.0489 - learning_rate: 0.1000 - val_custom_mse: 0.1356 - val_custom_mae: 0.2612
Epoch 5/100
1040/1040 - 9s - 8ms/step - loss: 0.0616 - mae: 0.1506 - mse: 0.0616 - val_loss: 0.0431 - val_mae: 0.1384 - val_mse: 0.0431 - learning_rate: 0.1000 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1040/1040 - 13s - 13ms/step - loss: 0.2486 - mae: 0.2907 - mse: 0.2486 - val_loss: 0.1077 - val_mae: 0.2359 - val_mse: 0.1077 - learning_rate: 0.1000 - val_custom_mse: 0.1642 - val_custom_mae: 0.2888
Epoch 2/100
1040/1040 - 8s - 8ms/step - loss: 0.1681 - mae: 0.2470 - mse: 0.1681 - val_loss: 0.0762 - val_mae: 0.1961 - val_mse: 0.0762 - learning_rate: 0.1000 - val_custom_mse: 0.1511 - val_custom_mae: 0.2771
Epoch 3/100
1040/1040 - 8s - 8ms/step - loss: 0.1051 - mae: 0.2010 - mse: 0.1051 - val_loss: 0.0576 - val_mae: 0.1668 - val_mse: 0.0576 - learning_rate: 0.1000 - val_custom_mse: 0.1418 - val_custom_mae: 0.2681
Epoch 4/100
1040/1040 - 8s - 8ms/step - loss: 0.0757 - mae: 0.1704 - mse: 0.0757 - val_loss: 0.0486 - val_mae: 0.1497 - val_mse: 0.0486 - learning_rate: 0.1000 - val_custom_mse: 0.1353 - val_custom_mae: 0.2608
Epoch 5/100
1040/1040 - 8s - 8ms/step - loss: 0.0635 - mae: 0.1531 - mse: 0.0635 - val_loss: 0.0432 - val_mae: 0.1387 - val_mse: 0.0432 - learning_rate: 0.1000 - val_cust

...

1020/1020 - 13s - 12ms/step - loss: 0.4666 - mae: 0.4057 - mse: 0.4666 - val_loss: 0.2524 - val_mae: 0.3454 - val_mse: 0.2524 - learning_rate: 0.1000 - val_custom_mse: 0.3429 - val_custom_mae: 0.4084
Epoch 2/100
1020/1020 - 8s - 8ms/step - loss: 0.3911 - mae: 0.3642 - mse: 0.3911 - val_loss: 0.2148 - val_mae: 0.3130 - val_mse: 0.2148 - learning_rate: 0.1000 - val_custom_mse: 0.3086 - val_custom_mae: 0.3885
Epoch 3/100
1020/1020 - 8s - 8ms/step - loss: 0.3522 - mae: 0.3329 - mse: 0.3522 - val_loss: 0.1961 - val_mae: 0.2912 - val_mse: 0.1961 - learning_rate: 0.1000 - val_custom_mse: 0.2918 - val_custom_mae: 0.3758
Epoch 4/100
1020/1020 - 8s - 8ms/step - loss: 0.3360 - mae: 0.3162 - mse: 0.3360 - val_loss: 0.1882 - val_mae: 0.2812 - val_mse: 0.1882 - learning_rate: 0.1000 - val_custom_mse: 0.2844 - val_custom_mae: 0.3699
Epoch 5/100
1020/1020 - 8s - 8ms/step - loss: 0.3282 - mae: 0.3070 - mse: 0.3282 - val_loss: 0.1838 - val_mae: 0.2741 - val_mse: 0.1838 - learning_rate: 0.1000 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_30', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1020/1020 - 13s - 13ms/step - loss: 0.4679 - mae: 0.4060 - mse: 0.4679 - val_loss: 0.2494 - val_mae: 0.3450 - val_mse: 0.2494 - learning_rate: 0.1000 - val_custom_mse: 0.3363 - val_custom_mae: 0.4058
Epoch 2/100
1020/1020 - 8s - 8ms/step - loss: 0.3914 - mae: 0.3640 - mse: 0.3914 - val_loss: 0.2125 - val_mae: 0.3102 - val_mse: 0.2125 - learning_rate: 0.1000 - val_custom_mse: 0.3068 - val_custom_mae: 0.3871
Epoch 3/100
1020/1020 - 8s - 8ms/step - loss: 0.3522 - mae: 0.3324 - mse: 0.3522 - val_loss: 0.1965 - val_mae: 0.2909 - val_mse: 0.1965 - learning_rate: 0.1000 - val_custom_mse: 0.2930 - val_custom_mae: 0.3765
Epoch 4/100
1020/1020 - 8s - 8ms/step - loss: 0.3371 - mae: 0.3168 - mse: 0.3371 - val_loss: 0.1878 - val_mae: 0.2803 - val_mse: 0.1878 - learning_rate: 0.1000 - val_custom_mse: 0.2845 - val_custom_mae: 0.3698
Epoch 5/100
1020/1020 - 8s - 8ms/step - loss: 0.3299 - mae: 0.3083 - mse: 0.3299 - val_loss: 0.1831 - val_mae: 0.2748 - val_mse: 0.1831 - learning_rate: 0.1000 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_31', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1020/1020 - 13s - 12ms/step - loss: 0.4692 - mae: 0.4071 - mse: 0.4692 - val_loss: 0.2509 - val_mae: 0.3455 - val_mse: 0.2509 - learning_rate: 0.1000 - val_custom_mse: 0.3391 - val_custom_mae: 0.4071
Epoch 2/100
1020/1020 - 8s - 8ms/step - loss: 0.3921 - mae: 0.3646 - mse: 0.3921 - val_loss: 0.2136 - val_mae: 0.3113 - val_mse: 0.2136 - learning_rate: 0.1000 - val_custom_mse: 0.3082 - val_custom_mae: 0.3881
Epoch 3/100
1020/1020 - 8s - 8ms/step - loss: 0.3537 - mae: 0.3336 - mse: 0.3537 - val_loss: 0.1968 - val_mae: 0.2914 - val_mse: 0.1968 - learning_rate: 0.1000 - val_custom_mse: 0.2935 - val_custom_mae: 0.3770
Epoch 4/100
1020/1020 - 8s - 8ms/step - loss: 0.3388 - mae: 0.3185 - mse: 0.3388 - val_loss: 0.1894 - val_mae: 0.2816 - val_mse: 0.1894 - learning_rate: 0.1000 - val_custom_mse: 0.2866 - val_custom_mae: 0.3713
Epoch 5/100
1020/1020 - 8s - 8ms/step - loss: 0.3321 - mae: 0.3106 - mse: 0.3321 - val_loss: 0.1869 - val_mae: 0.2783 - val_mse: 0.1869 - learning_rate: 0.1000 - val_cust

## Weather

In [ ]:
# Run the experiments
data_name='Weather'
results = run_experiments(data_name, horizons=[96,192,336,720], dropout_rates=[0.0, 0.1], revin=2, seq_len_=1024, learning_rate=1e-3, mopt='adamw')

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_32', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


1118/1118 - 14s - 12ms/step - loss: 0.3345 - mae: 0.2155 - mse: 0.3345 - val_loss: 0.0834 - val_mae: 0.0900 - val_mse: 0.0834 - learning_rate: 0.0010 - val_custom_mse: 0.4032 - val_custom_mae: 0.2912
Epoch 2/100
1118/1118 - 8s - 7ms/step - loss: 0.1150 - mae: 0.0832 - mse: 0.1150 - val_loss: 0.0599 - val_mae: 0.0662 - val_mse: 0.0599 - learning_rate: 0.0010 - val_custom_mse: 0.3883 - val_custom_mae: 0.2828
Epoch 3/100
1118/1118 - 8s - 7ms/step - loss: 0.0925 - mae: 0.0687 - mse: 0.0925 - val_loss: 0.0525 - val_mae: 0.0582 - val_mse: 0.0525 - learning_rate: 0.0010 - val_custom_mse: 0.3862 - val_custom_mae: 0.2802
Epoch 4/100
1118/1118 - 8s - 7ms/step - loss: 0.0776 - mae: 0.0606 - mse: 0.0776 - val_loss: 0.0473 - val_mae: 0.0521 - val_mse: 0.0473 - learning_rate: 0.0010 - val_custom_mse: 0.3854 - val_custom_mae: 0.2790
Epoch 5/100
1118/1118 - 8s - 7ms/step - loss: 0.0656 - mae: 0.0539 - mse: 0.0656 - val_loss: 0.0432 - val_mae: 0.0469 - val_mse: 0.0432 - learning_rate: 0.0010 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_33', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1118/1118 - 14s - 12ms/step - loss: 0.3646 - mae: 0.2474 - mse: 0.3646 - val_loss: 0.0826 - val_mae: 0.0887 - val_mse: 0.0826 - learning_rate: 0.0010 - val_custom_mse: 0.3999 - val_custom_mae: 0.2884
Epoch 2/100
1118/1118 - 8s - 7ms/step - loss: 0.1255 - mae: 0.1127 - mse: 0.1255 - val_loss: 0.0618 - val_mae: 0.0671 - val_mse: 0.0618 - learning_rate: 0.0010 - val_custom_mse: 0.3853 - val_custom_mae: 0.2794
Epoch 3/100
1118/1118 - 8s - 7ms/step - loss: 0.1068 - mae: 0.0996 - mse: 0.1068 - val_loss: 0.0555 - val_mae: 0.0618 - val_mse: 0.0555 - learning_rate: 0.0010 - val_custom_mse: 0.3854 - val_custom_mae: 0.2792
Epoch 4/100
1118/1118 - 8s - 7ms/step - loss: 0.0973 - mae: 0.0957 - mse: 0.0973 - val_loss: 0.0507 - val_mae: 0.0580 - val_mse: 0.0507 - learning_rate: 0.0010 - val_custom_mse: 0.3823 - val_custom_mae: 0.2761
Epoch 5/100
1118/1118 - 8s - 7ms/step - loss: 0.0907 - mae: 0.0936 - mse: 0.0907 - val_loss: 0.0474 - val_mae: 0.0558 - val_mse: 0.0474 - learning_rate: 0.0010 - val_cust

...

1111/1111 - 12s - 11ms/step - loss: 0.4791 - mae: 0.3163 - mse: 0.4791 - val_loss: 0.2032 - val_mae: 0.1694 - val_mse: 0.2032 - learning_rate: 0.0010 - val_custom_mse: 0.5148 - val_custom_mae: 0.3646
Epoch 2/100
1111/1111 - 7s - 7ms/step - loss: 0.2335 - mae: 0.1892 - mse: 0.2335 - val_loss: 0.1816 - val_mae: 0.1495 - val_mse: 0.1816 - learning_rate: 0.0010 - val_custom_mse: 0.5000 - val_custom_mae: 0.3568
Epoch 3/100
1111/1111 - 7s - 7ms/step - loss: 0.2159 - mae: 0.1766 - mse: 0.2159 - val_loss: 0.1757 - val_mae: 0.1449 - val_mse: 0.1757 - learning_rate: 0.0010 - val_custom_mse: 0.4985 - val_custom_mae: 0.3560
Epoch 4/100
1111/1111 - 7s - 7ms/step - loss: 0.2071 - mae: 0.1730 - mse: 0.2071 - val_loss: 0.1717 - val_mae: 0.1420 - val_mse: 0.1717 - learning_rate: 0.0010 - val_custom_mse: 0.4966 - val_custom_mae: 0.3542
Epoch 5/100
1111/1111 - 7s - 7ms/step - loss: 0.2030 - mae: 0.1712 - mse: 0.2030 - val_loss: 0.1699 - val_mae: 0.1414 - val_mse: 0.1699 - learning_rate: 0.0010 - val_cust

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_38', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


1099/1099 - 10s - 9ms/step - loss: 0.5681 - mae: 0.3675 - mse: 0.5681 - val_loss: 0.4362 - val_mae: 0.3119 - val_mse: 0.4362 - learning_rate: 0.0010 - val_custom_mse: 0.6018 - val_custom_mae: 0.4125
Epoch 2/100
1099/1099 - 7s - 6ms/step - loss: 0.4221 - mae: 0.3110 - mse: 0.4221 - val_loss: 0.4226 - val_mae: 0.3022 - val_mse: 0.4226 - learning_rate: 0.0010 - val_custom_mse: 0.5929 - val_custom_mae: 0.4093
Epoch 3/100
1099/1099 - 7s - 6ms/step - loss: 0.4099 - mae: 0.3042 - mse: 0.4099 - val_loss: 0.4194 - val_mae: 0.2995 - val_mse: 0.4194 - learning_rate: 0.0010 - val_custom_mse: 0.5918 - val_custom_mae: 0.4080
Epoch 4/100
1099/1099 - 7s - 6ms/step - loss: 0.4028 - mae: 0.3009 - mse: 0.4028 - val_loss: 0.4167 - val_mae: 0.2970 - val_mse: 0.4167 - learning_rate: 0.0010 - val_custom_mse: 0.5900 - val_custom_mae: 0.4066
Epoch 5/100
1099/1099 - 7s - 6ms/step - loss: 0.3981 - mae: 0.2983 - mse: 0.3981 - val_loss: 0.4151 - val_mae: 0.2951 - val_mse: 0.4151 - learning_rate: 0.0010 - val_custo

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'flow_mixer_39', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1099/1099 - 12s - 11ms/step - loss: 0.6012 - mae: 0.3873 - mse: 0.6012 - val_loss: 0.4351 - val_mae: 0.3120 - val_mse: 0.4351 - learning_rate: 0.0010 - val_custom_mse: 0.5995 - val_custom_mae: 0.4128
Epoch 2/100
1099/1099 - 7s - 7ms/step - loss: 0.4285 - mae: 0.3212 - mse: 0.4285 - val_loss: 0.4239 - val_mae: 0.3031 - val_mse: 0.4239 - learning_rate: 0.0010 - val_custom_mse: 0.5933 - val_custom_mae: 0.4099
Epoch 3/100
1099/1099 - 7s - 7ms/step - loss: 0.4177 - mae: 0.3145 - mse: 0.4177 - val_loss: 0.4206 - val_mae: 0.3022 - val_mse: 0.4206 - learning_rate: 0.0010 - val_custom_mse: 0.5913 - val_custom_mae: 0.4091
Epoch 4/100
1099/1099 - 7s - 7ms/step - loss: 0.4132 - mae: 0.3131 - mse: 0.4132 - val_loss: 0.4180 - val_mae: 0.3008 - val_mse: 0.4180 - learning_rate: 0.0010 - val_custom_mse: 0.5892 - val_custom_mae: 0.4076
Epoch 5/100
1099/1099 - 7s - 7ms/step - loss: 0.4106 - mae: 0.3122 - mse: 0.4106 - val_loss: 0.4172 - val_mae: 0.3003 - val_mse: 0.4172 - learning_rate: 0.0010 - val_cust

## Electricity

In [ ]:
# Run the experiments
data_name='ECL'
results = run_experiments(data_name,horizons=[96, 192, 336, 720], dropout_rates=[0.0],revin=1, learning_rate=1e-4, mopt='adamw', seq_len_=1024*3 )

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


477/477 - 146s - 306ms/step - loss: 0.3271 - mae: 0.3460 - mse: 0.3271 - val_loss: 0.3069 - val_mae: 0.3261 - val_mse: 0.3069 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2746 - val_custom_mae: 0.3076
Epoch 2/100
477/477 - 138s - 290ms/step - loss: 0.2570 - mae: 0.3206 - mse: 0.2570 - val_loss: 0.2501 - val_mae: 0.3081 - val_mse: 0.2501 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2341 - val_custom_mae: 0.2962
Epoch 3/100
477/477 - 130s - 273ms/step - loss: 0.2164 - mae: 0.3020 - mse: 0.2164 - val_loss: 0.2158 - val_mae: 0.2907 - val_mse: 0.2158 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2157 - val_custom_mae: 0.2878
Epoch 4/100
477/477 - 130s - 272ms/step - loss: 0.1871 - mae: 0.2828 - mse: 0.1871 - val_loss: 0.1866 - val_mae: 0.2721 - val_mse: 0.1866 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2014 - val_custom_mae: 0.2791
Epoch 5/100
477/477 - 130s - 272ms/step - loss: 0.1610 - mae: 0.2629 - mse: 0.1610 - val_loss: 0.1600 - val_mae: 0.2528 - val_mse: 0.1600 - learning

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


474/474 - 136s - 288ms/step - loss: 0.2988 - mae: 0.3344 - mse: 0.2988 - val_loss: 0.2907 - val_mae: 0.3172 - val_mse: 0.2907 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2587 - val_custom_mae: 0.2994
Epoch 2/100
474/474 - 130s - 273ms/step - loss: 0.2532 - mae: 0.3140 - mse: 0.2532 - val_loss: 0.2446 - val_mae: 0.2995 - val_mse: 0.2446 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2359 - val_custom_mae: 0.2927
Epoch 3/100
474/474 - 129s - 272ms/step - loss: 0.2144 - mae: 0.2950 - mse: 0.2144 - val_loss: 0.2095 - val_mae: 0.2835 - val_mse: 0.2095 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2202 - val_custom_mae: 0.2872
Epoch 4/100
474/474 - 130s - 273ms/step - loss: 0.1844 - mae: 0.2772 - mse: 0.1844 - val_loss: 0.1818 - val_mae: 0.2683 - val_mse: 0.1818 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2083 - val_custom_mae: 0.2825
Epoch 5/100
474/474 - 129s - 272ms/step - loss: 0.1597 - mae: 0.2601 - mse: 0.1597 - val_loss: 0.1576 - val_mae: 0.2515 - val_mse: 0.1576 - learning

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


469/469 - 148s - 316ms/step - loss: 0.2517 - mae: 0.3112 - mse: 0.2517 - val_loss: 0.2460 - val_mae: 0.2956 - val_mse: 0.2460 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2176 - val_custom_mae: 0.2802
Epoch 2/100
469/469 - 128s - 272ms/step - loss: 0.2206 - mae: 0.2936 - mse: 0.2206 - val_loss: 0.2183 - val_mae: 0.2805 - val_mse: 0.2183 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2090 - val_custom_mae: 0.2754
Epoch 3/100
469/469 - 127s - 271ms/step - loss: 0.1925 - mae: 0.2765 - mse: 0.1925 - val_loss: 0.1926 - val_mae: 0.2662 - val_mse: 0.1926 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2016 - val_custom_mae: 0.2712
Epoch 4/100
469/469 - 127s - 271ms/step - loss: 0.1690 - mae: 0.2613 - mse: 0.1690 - val_loss: 0.1698 - val_mae: 0.2523 - val_mse: 0.1698 - learning_rate: 1.0000e-04 - val_custom_mse: 0.1948 - val_custom_mae: 0.2674
Epoch 5/100
469/469 - 127s - 271ms/step - loss: 0.1491 - mae: 0.2470 - mse: 0.1491 - val_loss: 0.1495 - val_mae: 0.2386 - val_mse: 0.1495 - learning

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


457/457 - 128s - 281ms/step - loss: 0.4489 - mae: 0.4314 - mse: 0.4489 - val_loss: 0.4129 - val_mae: 0.4022 - val_mse: 0.4129 - learning_rate: 1.0000e-04 - val_custom_mse: 0.3773 - val_custom_mae: 0.3790
Epoch 2/100
457/457 - 122s - 267ms/step - loss: 0.3507 - mae: 0.3907 - mse: 0.3507 - val_loss: 0.3241 - val_mae: 0.3678 - val_mse: 0.3241 - learning_rate: 1.0000e-04 - val_custom_mse: 0.3252 - val_custom_mae: 0.3631
Epoch 3/100
457/457 - 122s - 266ms/step - loss: 0.2874 - mae: 0.3601 - mse: 0.2874 - val_loss: 0.2744 - val_mae: 0.3445 - val_mse: 0.2744 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2981 - val_custom_mae: 0.3526
Epoch 4/100
457/457 - 122s - 267ms/step - loss: 0.2494 - mae: 0.3372 - mse: 0.2494 - val_loss: 0.2419 - val_mae: 0.3262 - val_mse: 0.2419 - learning_rate: 1.0000e-04 - val_custom_mse: 0.2809 - val_custom_mae: 0.3443
Epoch 5/100
457/457 - 122s - 268ms/step - loss: 0.2215 - mae: 0.3177 - mse: 0.2215 - val_loss: 0.2151 - val_mae: 0.3078 - val_mse: 0.2151 - learning

## Traffic

In [ ]:
# Run the experiments
data_name='TrafficL'
results = run_experiments(data_name,horizons=[96, 192, 336, 720], dropout_rates=[0.0],revin=2, learning_rate=1e-3, mopt='adamw', seq_len_=4096 )

# Print final results
print(f"\n {data_name} Final Results:")
df = pd.DataFrame(results)
# Assuming results is a list of dictionaries with horizon, dropout, MSE, and MAE values
display_results_tables(results[0])


# The results are already saved in CSV format after each experiment
print(f"\nResults saved to: ./flowmixer_results/{data_name}_experiment_results.csv")

Running experiment: horizon=96, dropout_rate=0.0
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


253/253 - 103s - 409ms/step - loss: 0.5703 - mae: 0.4043 - mse: 0.5703 - val_loss: 0.3984 - val_mae: 0.3391 - val_mse: 0.3984 - learning_rate: 0.0010 - val_custom_mse: 0.6558 - val_custom_mae: 0.3788
Epoch 2/100
253/253 - 92s - 364ms/step - loss: 0.2483 - mae: 0.2732 - mse: 0.2483 - val_loss: 0.1789 - val_mae: 0.2289 - val_mse: 0.1789 - learning_rate: 0.0010 - val_custom_mse: 0.4775 - val_custom_mae: 0.3116
Epoch 3/100
253/253 - 92s - 364ms/step - loss: 0.1072 - mae: 0.1789 - mse: 0.1072 - val_loss: 0.0810 - val_mae: 0.1526 - val_mse: 0.0810 - learning_rate: 0.0010 - val_custom_mse: 0.3917 - val_custom_mae: 0.2752
Epoch 4/100
253/253 - 92s - 364ms/step - loss: 0.0496 - mae: 0.1199 - mse: 0.0496 - val_loss: 0.0402 - val_mae: 0.1041 - val_mse: 0.0402 - learning_rate: 0.0010 - val_custom_mse: 0.3520 - val_custom_mae: 0.2537
Epoch 5/100
253/253 - 92s - 363ms/step - loss: 0.0254 - mae: 0.0827 - mse: 0.0254 - val_loss: 0.0226 - val_mae: 0.0732 - val_mse: 0.0226 - learning_rate: 0.0010 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


250/250 - 101s - 403ms/step - loss: 0.4454 - mae: 0.3253 - mse: 0.4454 - val_loss: 0.3439 - val_mae: 0.2869 - val_mse: 0.3439 - learning_rate: 0.0010 - val_custom_mse: 0.5392 - val_custom_mae: 0.3169
Epoch 2/100
250/250 - 90s - 358ms/step - loss: 0.2127 - mae: 0.2328 - mse: 0.2127 - val_loss: 0.1469 - val_mae: 0.1953 - val_mse: 0.1469 - learning_rate: 0.0010 - val_custom_mse: 0.4136 - val_custom_mae: 0.2736
Epoch 3/100
250/250 - 90s - 359ms/step - loss: 0.0888 - mae: 0.1546 - mse: 0.0888 - val_loss: 0.0666 - val_mae: 0.1298 - val_mse: 0.0666 - learning_rate: 0.0010 - val_custom_mse: 0.3587 - val_custom_mae: 0.2506
Epoch 4/100
250/250 - 90s - 359ms/step - loss: 0.0416 - mae: 0.1030 - mse: 0.0416 - val_loss: 0.0360 - val_mae: 0.0882 - val_mse: 0.0360 - learning_rate: 0.0010 - val_custom_mse: 0.3369 - val_custom_mae: 0.2388
Epoch 5/100
250/250 - 90s - 359ms/step - loss: 0.0236 - mae: 0.0710 - mse: 0.0236 - val_loss: 0.0239 - val_mae: 0.0619 - val_mse: 0.0239 - learning_rate: 0.0010 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


246/246 - 99s - 402ms/step - loss: 0.3560 - mae: 0.2507 - mse: 0.3560 - val_loss: 0.2895 - val_mae: 0.2293 - val_mse: 0.2895 - learning_rate: 0.0010 - val_custom_mse: 0.4560 - val_custom_mae: 0.2563
Epoch 2/100
246/246 - 87s - 353ms/step - loss: 0.1859 - mae: 0.1947 - mse: 0.1859 - val_loss: 0.1314 - val_mae: 0.1667 - val_mse: 0.1314 - learning_rate: 0.0010 - val_custom_mse: 0.3800 - val_custom_mae: 0.2418
Epoch 3/100
246/246 - 88s - 356ms/step - loss: 0.0795 - mae: 0.1322 - mse: 0.0795 - val_loss: 0.0635 - val_mae: 0.1111 - val_mse: 0.0635 - learning_rate: 0.0010 - val_custom_mse: 0.3528 - val_custom_mae: 0.2354
Epoch 4/100
246/246 - 89s - 361ms/step - loss: 0.0403 - mae: 0.0886 - mse: 0.0403 - val_loss: 0.0404 - val_mae: 0.0762 - val_mse: 0.0404 - learning_rate: 0.0010 - val_custom_mse: 0.3468 - val_custom_mae: 0.2355
Epoch 5/100
246/246 - 88s - 359ms/step - loss: 0.0274 - mae: 0.0622 - mse: 0.0274 - val_loss: 0.0327 - val_mae: 0.0553 - val_mse: 0.0327 - learning_rate: 0.0010 - val_c

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_custom_mse` which is not available. Available metrics are: loss,mae,mse,val_loss,val_mae,val_mse
  current = self.get_monitor_value(logs)


234/234 - 89s - 382ms/step - loss: 0.6009 - mae: 0.4182 - mse: 0.6009 - val_loss: 0.4552 - val_mae: 0.3613 - val_mse: 0.4552 - learning_rate: 0.0010 - val_custom_mse: 0.6672 - val_custom_mae: 0.3894
Epoch 2/100
234/234 - 78s - 333ms/step - loss: 0.2965 - mae: 0.2968 - mse: 0.2965 - val_loss: 0.2451 - val_mae: 0.2586 - val_mse: 0.2451 - learning_rate: 0.0010 - val_custom_mse: 0.5276 - val_custom_mae: 0.3350
Epoch 3/100
234/234 - 78s - 333ms/step - loss: 0.1584 - mae: 0.2116 - mse: 0.1584 - val_loss: 0.1505 - val_mae: 0.1892 - val_mse: 0.1505 - learning_rate: 0.0010 - val_custom_mse: 0.4581 - val_custom_mae: 0.3001
Epoch 4/100
234/234 - 78s - 334ms/step - loss: 0.0992 - mae: 0.1574 - mse: 0.0992 - val_loss: 0.1077 - val_mae: 0.1434 - val_mse: 0.1077 - learning_rate: 0.0010 - val_custom_mse: 0.4225 - val_custom_mae: 0.2771
Epoch 5/100
234/234 - 78s - 333ms/step - loss: 0.0725 - mae: 0.1224 - mse: 0.0725 - val_loss: 0.0877 - val_mae: 0.1135 - val_mse: 0.0877 - learning_rate: 0.0010 - val_c

In [ ]:
display_results_tables(results[0]) # Traffic Dataset


MSE Results:
          DR=0%
Horizon        
96       0.3771
192      0.3886
336      0.4015
720      0.4343

MAE Results:
          DR=0%
Horizon        
96       0.2641
192      0.2684
336      0.2748
720      0.2919
